## Look at the Data


In [106]:
# Import necesaary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import cvxpy as cp
import time

# Ensure reproducability
np.random.seed(0)

In [107]:
# Load the .mat file
data = loadmat("Data/mimo_detection.mat")

# Look at data
for  key, item in data.items():
    if type(item) != np.ndarray:
        print(f"{key}: {item}")
    else:
        print(f"{key}: shape {np.shape(item)}")
        print(f" Complex: {item.dtype.kind == 'c'}")
print("\n")

# Extract variables
Nrx = data['Nrx'][0][0]   # Number of receive antennas
print(f'Number of receive antennas: {Nrx}')
Ntx = data['Ntx'][0][0]  # Number of transmit antennas
print(f'Number of transmit antennas: {Ntx}')
SNR = data['snrdB'][0][0]
print(f'SNR: {SNR} dB')
Sc = data['sc']  # Transmitted symbols
Hc = data['Hc']  # Channel matrix
Yc = data['yc']  # Received symbols


__header__: b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Thu Dec  1 07:49:52 2016'
__version__: 1.0
__globals__: []
Hc: shape (40, 40)
 Complex: True
Nrx: shape (1, 1)
 Complex: False
Ntx: shape (1, 1)
 Complex: False
sc: shape (40, 1)
 Complex: True
snrdB: shape (1, 1)
 Complex: False
yc: shape (40, 1)
 Complex: True


Number of receive antennas: 40
Number of transmit antennas: 40
SNR: 20 dB


## Convert to appropriate form for optimization

In [108]:
# Make it a real-valued problem
H_real = np.real(Hc)
H_imag = np.imag(Hc)
H = np.block([[H_real, -H_imag],
              [H_imag, H_real]])
Y_real = np.real(Yc)
Y_imag = np.imag(Yc)
y = np.vstack((Y_real, Y_imag))
Sc_real = np.real(Sc)
Sc_imag = np.imag(Sc)
s_labels = np.vstack((Sc_real, Sc_imag))


# Make the matrices for the QP formulation
HtH = H.T @ H
Hty = H.T @ y
yty = y.T @ y      # this is already shape (1,1)
A = np.block([
    [HtH,     -Hty],
    [-Hty.T,  yty]
])



## Optimize using CVXPy Library

In [109]:
# Parameters
N = 2 * Ntx + 1
eps = 1e-5
max_iters = 50000

# Define and solve the SDP relaxation
X = cp.Variable((N, N), PSD=True)
constraints = [cp.diag(X) == 1]
objective = cp.Minimize(cp.trace(A @ X))
prob = cp.Problem(objective, constraints)

# Solve
t0 = time.perf_counter()
prob.solve(solver=cp.CVXOPT, verbose=True)
t1 = time.perf_counter()

X_star = X.value

# Get results
print("status:", prob.status)
print("objective:", prob.value)
print(f"wall-clock solve time: {t1 - t0:.3f} s")

# Solver metadata 
# CVXPY stores solver-specific statistics here when available.
stats = prob.solver_stats
print("\n--- Solver stats (CVXPY) ---")
print("solver name:", stats.solver_name)
print("solve time reported by solver (s):", stats.solve_time)
print("setup time reported by solver (s):", stats.setup_time)
print("num iterations:", stats.num_iters)

# Check feasibility / quality on X_star 
print("\n--- Solution checks ---")

# Symmetry check
sym_err = np.linalg.norm(X_star - X_star.T, ord='fro') / max(1.0, np.linalg.norm(X_star, ord='fro'))
print(f"relative symmetry error: {sym_err:.2e}")

# Enforce symmetry for eigen-checks
X_sym = 0.5 * (X_star + X_star.T)

# Diagonal constraint satisfaction
diag_err = np.linalg.norm(np.diag(X_sym) - 1, ord=np.inf)
print(f"diag constraint max error ||diag(X)-1||_inf: {diag_err:.2e}")

# PSD check 
eigvals = np.linalg.eigvalsh(X_sym)
min_eig = eigvals.min()
print(f"min eigenvalue of X (symmetrized): {min_eig:.2e}")

# Rank-indicator / tightness check
trace_X = np.trace(X_sym)
top_frac = eigvals.max() / eigvals.sum()
print(f"trace(X): {trace_X:.6f} (should be ~{X_sym.shape[0]:d})")
print(f"largest-eigenvalue fraction lambda_max/sum(lambda): {top_frac:.4f}")

# Objective consistency: trace(A X) vs cvxpy value
obj_check = np.trace(A @ X_sym)
print(f"objective recomputed Tr(A X): {obj_check:.6e}")


(CVXPY) Jan 09 10:26:19 PM: Your problem has 6561 variables, 81 constraints, and 0 parameters.
(CVXPY) Jan 09 10:26:19 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 09 10:26:19 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 09 10:26:19 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 09 10:26:19 PM: Your problem is compiled with the CPP canonicalization backend.
(CVXPY) Jan 09 10:26:19 PM: Compiling problem (target solver=CVXOPT).
(CVXPY) Jan 09 10:26:19 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> CVXOPT
(CVXPY) Jan 09 10:26:19 PM: Applying reduction Dcp2Cone
(CVXPY) Jan 09 10:26:19 PM: Applying reduction CvxAttr2Constr
(CVXPY) Jan 09 10:26:19 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Jan 09 10:26:19 PM: Applying reduction CVXOPT
(CVXPY) Jan 09 10:26:19 PM: Finished problem compilation 

                                     CVXPY                                     
                                     v1.7.5                                    
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
     pcost       dcost       gap    pres   dres   k/t
 0:  1.5407e+04  1.5407e+04  1e+05  0e+00  1e+00  1e+00
 1:  1.1037e+04  1.1063e+04  1e+05  6e-14  1e+00  3e+01
 2:  1.1769e+04  1.2156e+04  6e+04  6e-14  8e-01  4e+02
 3:  8.6565e+03  8.7866e+03  2e+04  4e-14  2e-01  1e+02
 4:  6.8140e+03  6.9493e+03  2e+04  6e-14  2e-01  1e+02
 5:  3.4397e+03  3.4788e+0

(CVXPY) Jan 09 10:26:53 PM: Problem status: optimal
(CVXPY) Jan 09 10:26:53 PM: Optimal value: 3.092e+01
(CVXPY) Jan 09 10:26:53 PM: Compilation took 1.133e-01 seconds
(CVXPY) Jan 09 10:26:53 PM: Solver (including time spent in interface) took 3.434e+01 seconds


17:  3.0919e+01  3.0919e+01  1e-05  9e-11  2e-10  2e-07
Optimal solution found.
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
status: optimal
objective: 30.919463389238388
wall-clock solve time: 34.481 s

--- Solver stats (CVXPY) ---
solver name: CVXOPT
solve time reported by solver (s): None
setup time reported by solver (s): None
num iterations: None

--- Solution checks ---
relative symmetry error: 0.00e+00
diag constraint max error ||diag(X)-1||_inf: 2.22e-16
min eigenvalue of X (symmetrized): 6.13e-12
trace(X): 81.000000 (should be ~81)
largest-eigenvalue fraction lambda_max/sum(lambda): 0.9524
objective recomputed Tr(A X): 3.091946e+01


## Extracting a Rank 1 Matrix from the result

### Gaussian Randomization

In [110]:
def complex_to_real(Hc, yc):
    # yc can be (M,1) or (M,)
    yc = np.asarray(yc).reshape(-1)

    Hr = np.block([
        [Hc.real, -Hc.imag],
        [Hc.imag,  Hc.real]
    ])
    yr = np.concatenate([yc.real, yc.imag], axis=0)
    return Hr, yr

def local_refinement(s_init, H, y):
    """
    Greedy 1-bit local refinement for ±1 vectors.

    Parameters
    ----------
    s_init : ndarray
        Initial feasible ±1 vector.
    H : ndarray
        Channel matrix.
    y : ndarray
        Received signal.

    Returns
    -------
    s_best : ndarray
        Locally refined ±1 vector.
    best_cost : float
        Corresponding ML cost.
    """

    s_best = s_init.copy()
    best_cost = np.linalg.norm(y - H @ s_best)**2

    improved = True
    while improved:
        improved = False
        for i in range(len(s_best)):
            s_try = s_best.copy()
            s_try[i] *= -1  # flip one bit

            cost = np.linalg.norm(y - H @ s_try)**2
            if cost < best_cost:
                best_cost = cost
                s_best = s_try
                improved = True
                break  # restart scan (greedy)

    return s_best, best_cost


def gaussian_randomization(
    X_star, H, y, num_samples=500, eps=1e-8, refinement=0
):
    """
    Gaussian randomization for SDR-based detection, with optional local refinement.

    NOTE: Reproducibility is controlled by a global call to np.random.seed(...)
    made outside this function.
    """

    d = X_star.shape[0]          # d = 2N + 1
    N = d - 1

    # Eigen factorization: X* = U U^T
    eigvals, eigvecs = np.linalg.eigh(X_star)
    eigvals = np.maximum(eigvals, 0)
    U = eigvecs @ np.diag(np.sqrt(eigvals))

    best_cost = np.inf
    s_best = None

    for _ in range(num_samples):
        # Sample from N(0, X*)
        r = np.random.randn(d)
        xi = U @ r

        s_tilde = xi[:N]
        t_tilde = xi[N]

        if abs(t_tilde) < eps:
            continue

        s_hat = s_tilde / t_tilde
        s_candidate = np.sign(s_hat)
        s_candidate[s_candidate == 0] = 1

        if refinement == 1:
            s_candidate, cost = local_refinement(s_candidate, H, y)
        else:
            cost = np.linalg.norm(y - H @ s_candidate)**2

        if cost < best_cost:
            best_cost = cost
            s_best = s_candidate.copy()

    return s_best, best_cost


In [111]:
print("X_star:", X_star.shape)
print("Hc    :", Hc.shape)
print("Yc    :", Yc.shape)


X_star: (81, 81)
Hc    : (40, 40)
Yc    : (40, 1)


In [112]:
H, y = complex_to_real(Hc, Yc)

s_hat_g, cost_g = gaussian_randomization(X_star, H, y)

print("Best ML cost:", cost_g)
print("Detected symbols:")
print(s_hat_g.astype(int))

Best ML cost: 39.776746009731916
Detected symbols:
[-1 -1 -1 -1 -1  1 -1  1 -1 -1 -1  1  1  1  1 -1 -1 -1  1  1  1  1 -1  1
  1  1  1  1  1 -1 -1  1 -1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  1  1  1 -1 -1 -1  1 -1  1  1 -1  1  1 -1 -1  1  1 -1  1 -1 -1  1  1 -1
  1 -1 -1 -1 -1  1 -1  1]


#### Gaussian Randomization with Local Refinement

In [113]:
s_hat_gr, cost_gr = gaussian_randomization(X_star, H, y, refinement=1)

print("Best ML cost:", cost_gr)
print("Detected symbols:")
print(s_hat_gr.astype(int))

Best ML cost: 39.776746009731916
Detected symbols:
[-1 -1 -1 -1 -1  1 -1  1 -1 -1 -1  1  1  1  1 -1 -1 -1  1  1  1  1 -1  1
  1  1  1  1  1 -1 -1  1 -1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  1  1  1 -1 -1 -1  1 -1  1  1 -1  1  1 -1 -1  1  1 -1  1 -1 -1  1  1 -1
  1 -1 -1 -1 -1  1 -1  1]


### Sphere Randomization

In [114]:
def sphere_randomization(
    X_star, H, y, num_samples=500, eps=1e-8, refinement=0
):
    """
    Geometry-aware (sphere-normalized) randomization for SDR-based detection,
    with optional local refinement.

    NOTE: Reproducibility is controlled by a global call to np.random.seed(...)
    made outside this function.
    """

    d = X_star.shape[0]      # d = 2N + 1
    N = d - 1

    # Eigen factorization: X* = U U^T
    eigvals, eigvecs = np.linalg.eigh(X_star)
    eigvals = np.maximum(eigvals, 0)
    U = eigvecs @ np.diag(np.sqrt(eigvals))

    best_cost = np.inf
    s_best = None

    for _ in range(num_samples):
        # Sample a random direction on the unit sphere
        r = np.random.randn(d)
        norm_r = np.linalg.norm(r)
        if norm_r < eps:
            continue
        r /= norm_r

        xi = U @ r

        s_tilde = xi[:N]
        t_tilde = xi[N]

        if abs(t_tilde) < eps:
            continue

        s_hat = s_tilde / t_tilde
        s_candidate = np.sign(s_hat)
        s_candidate[s_candidate == 0] = 1

        if refinement == 1:
            s_candidate, cost = local_refinement(s_candidate, H, y)
        else:
            cost = np.linalg.norm(y - H @ s_candidate)**2

        if cost < best_cost:
            best_cost = cost
            s_best = s_candidate.copy()

    return s_best, best_cost



In [115]:
s_hat_s, cost_s = sphere_randomization(X_star, H, y, refinement=0)  # baseline sphere

print("Best ML cost:", cost_s)
print("Detected symbols:")
print(s_hat_s.astype(int))

Best ML cost: 39.776746009731916
Detected symbols:
[-1 -1 -1 -1 -1  1 -1  1 -1 -1 -1  1  1  1  1 -1 -1 -1  1  1  1  1 -1  1
  1  1  1  1  1 -1 -1  1 -1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  1  1  1 -1 -1 -1  1 -1  1  1 -1  1  1 -1 -1  1  1 -1  1 -1 -1  1  1 -1
  1 -1 -1 -1 -1  1 -1  1]


### Sphere Randomization with Local Refinement

In [116]:
s_hat_sr, cost_sr = sphere_randomization(X_star, H, y, refinement=0)  # baseline sphere

print("Best ML cost:", cost_sr)
print("Detected symbols:")
print(s_hat_sr.astype(int))

Best ML cost: 39.776746009731916
Detected symbols:
[-1 -1 -1 -1 -1  1 -1  1 -1 -1 -1  1  1  1  1 -1 -1 -1  1  1  1  1 -1  1
  1  1  1  1  1 -1 -1  1 -1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  1  1  1 -1 -1 -1  1 -1  1  1 -1  1  1 -1 -1  1  1 -1  1 -1 -1  1  1 -1
  1 -1 -1 -1 -1  1 -1  1]


### Error Metrics

In [ ]:
def realstack_to_qpsk(s_hat, Sc):
    """
    Convert real-stacked ±1 vector [Re; Im] to complex QPSK symbols.
    """
    Sc = np.asarray(Sc).reshape(-1)
    s_hat = np.asarray(s_hat).reshape(-1)

    N = Sc.size
    assert s_hat.size == 2 * N, "Dimension mismatch in real-stacked solution"

    sR = s_hat[:N]
    sI = s_hat[N:]
    s_complex = sR + 1j * sI

    return s_complex, sR, sI

def compute_ser_ber(s_detected, sR, sI, Sc):
    Sc = np.asarray(Sc).reshape(-1)

    # Quantize both to QPSK constellation {±1 ± j}
    Sc_q = np.sign(Sc.real) + 1j*np.sign(Sc.imag)
    sd_q = np.sign(s_detected.real) + 1j*np.sign(s_detected.imag)

    # Symbol Error Rate
    SER = np.mean(sd_q != Sc_q)

    # Bit Error Rate (2 bits per symbol)
    bit_errors = np.sum(np.sign(sR) != np.sign(Sc.real)) \
               + np.sum(np.sign(sI) != np.sign(Sc.imag))
    BER = bit_errors / (2 * Sc.size)

    return SER, BER





In [118]:
def timed_randomization(method, *args, **kwargs):
    """
    Time a randomization method (Gaussian or Sphere).
    """
    t0 = time.perf_counter()
    s_hat, cost = method(*args, **kwargs)
    t1 = time.perf_counter()

    runtime = t1 - t0
    return s_hat, cost, runtime


In [119]:
s_hat_g, cost_g, time_g = timed_randomization(
    gaussian_randomization,
    X_star, H, y,
    num_samples=500,
    refinement=0
)

s_hat_gr, cost_gr, time_gr = timed_randomization(
    gaussian_randomization,
    X_star, H, y,
    num_samples=500,
    refinement=1
)

s_hat_s, cost_s, time_s = timed_randomization(
    sphere_randomization,
    X_star, H, y,
    num_samples=500,
    refinement=0
)

s_hat_sr, cost_sr, time_sr = timed_randomization(
    sphere_randomization,
    X_star, H, y,
    num_samples=500,
    refinement=1
)


In [120]:
results = {}

results["Gaussian"] = (cost_g, SER_g, BER_g, time_g)
results["Gaussian + Refinement"] = (cost_gr, SER_gr, BER_gr, time_gr)
results["Sphere"] = (cost_s, SER_s, BER_s, time_s)
results["Sphere + Refinement"] = (cost_sr, SER_sr, BER_sr, time_sr)

for k, (cost, ser, ber, t) in results.items():
    print(
        f"{k:25s} | "
        f"ML cost = {cost:.3e} | "
        f"SER = {ser:.4f} | "
        f"BER = {ber:.4f} | "
        f"time = {t:.3e} s"
    )



Gaussian                  | ML cost = 3.978e+01 | SER = 0.0000 | BER = 0.0000 | time = 1.359e-02 s
Gaussian + Refinement     | ML cost = 3.978e+01 | SER = 0.0000 | BER = 0.0000 | time = 6.188e-01 s
Sphere                    | ML cost = 3.978e+01 | SER = 0.0000 | BER = 0.0000 | time = 6.300e-03 s
Sphere + Refinement       | ML cost = 3.978e+01 | SER = 0.0000 | BER = 0.0000 | time = 6.594e-01 s
